In [ ]:
%load_ext autoreload
%autoreload 2
import os
from functools import partial
from dataclasses import dataclass
from typing import Optional, Callable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from pmf import PoissonMF

# Load & Process Data

In [ ]:
data_path = '/scratch/sm2537/data/03_13_24'
notes = pd.read_csv(os.path.join(data_path, 'notes-00000.tsv'), sep='\t')
# Convert NaN to empty string
notes['summary'] = notes['summary'].astype(str).fillna('').str.strip()

# read in ratings from 'ratings-00000.tsv' to 'ratings-00007.tsv'
# and concatenate them into a single DataFrame
for i in range(8):
    print(i)
    filepath = os.path.join(data_path, f'ratings-0000{i}.tsv')
    if i == 0:
        ratings = pd.read_csv(filepath, sep='\t')
    else:
        ratings = pd.concat([ratings, pd.read_csv(filepath, sep='\t')])

# Drop rows with NaN in helpfulnessLevel column
ratings = ratings.dropna(subset=['helpfulnessLevel'])

In [ ]:
note_status_df = pd.read_csv(os.path.join(data_path, 'noteStatusHistory-00000.tsv'), sep='\t')

In [ ]:
# Print total number of ratings
print('Total number of ratings: {}'.format(len(ratings)))

# Print number of unique notes and raters
print('Number of unique notes: {}'.format(ratings['noteId'].nunique()))
print('Number of unique raters: {}'.format(ratings['raterParticipantId'].nunique()))

# Get list of notes with more than 5 ratings
note_rating_counts = ratings['noteId'].value_counts()
filtered_note_ids = note_rating_counts[note_rating_counts > 5].index.tolist()
print('Number of notes with more than 5 ratings: {}'.format(len(notes)))

# Get list of raters with more than 10 ratings
rater_counts = ratings['raterParticipantId'].value_counts()
filtered_rater_ids = rater_counts[rater_counts > 10].index.tolist()
print('Number of raters with more than 10 ratings: {}'.format(len(filtered_rater_ids)))

# Filter ratings to only include ratings rated by raters with more than 10 ratings and for notes with more than 5 ratings
ratings = ratings[ratings['raterParticipantId'].isin(filtered_rater_ids) & ratings['noteId'].isin(filtered_note_ids)]
print('Number of ratings after filtering: {}'.format(len(ratings)))

In [ ]:
# Convert the ratings matrix to three lists:
# - rating_labels, which is the 'helpfulnessLevel' column mapped to -1 for 'NOT_HELPFUL',
#   0 for 'SOMEWHAT_HELPFUL', and 1 for 'HELPFUL'
# - user_idxs, which is the 'raterParticipantId' column mapped to a unique integer
# - note_idxs, which is the 'noteId' column mapped to a unique integer
rating_labels = ratings['helpfulnessLevel'].map({'NOT_HELPFUL': -1, 'SOMEWHAT_HELPFUL': 0, 'HELPFUL': 1})
# Use a label encoder to map the user and note ids to unique integers
user_encoder = LabelEncoder()
note_encoder = LabelEncoder()
user_idxs = user_encoder.fit_transform(ratings['raterParticipantId'])
note_idxs = note_encoder.fit_transform(ratings['noteId'])

n_users = len(user_encoder.classes_)
n_notes = len(note_encoder.classes_)

# Sparse exposure matrix (did the user rate the note?)
exp_matrix = csr_matrix((np.ones_like(rating_labels), (user_idxs, note_idxs)), shape=(n_users, n_notes))

# Step 1a: Causal Inference, Exposure Model
Fit Poisson matrix factorization to the exposures/assignments (who rated what). We will then use the reconstructed exposures as substitute confounders.

In [ ]:
# pf = PoissonMF(n_components=4, random_state=1, verbose=True, a=0.3, b=0.3, c=0.3, d=0.3)
# pf.fit(exp_matrix, user_idxs, note_idxs)
# Latent representations learned by Poisson MF
# exp_user_factors, exp_item_factors = pf.Eb, pf.Et.T

In [ ]:
# # save exp_user_factors and exp_item_factors
# np.save('exp_user_factors.npy', exp_user_factors)
# np.save('exp_item_factors.npy', exp_item_factors)

In [ ]:
# load exp_user_factors and exp_item_factors
exp_user_factors = np.load('out/exp_user_factors.npy')
exp_item_factors = np.load('out/exp_item_factors.npy')

# Step 1b: Causal Inference, Outcome Model
Now estimate the outcome model, i.e., matrix factorization on the observed ratings while controlling for the substitute confounders estimated from Step 1a.

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch import nn
from mf import MatrixFactorizationModel, ModelData

In [ ]:
n_components=4

# Our full model that deconfounds with the substitute confounder from step 1a
deconf_mf_model = MatrixFactorizationModel(
    n_users, n_notes, 
    exp_user_factors=exp_user_factors,
    exp_item_factors=exp_item_factors,
    n_components=n_components)

# Regular matrix factorization without deconfounding
mf_model = MatrixFactorizationModel(n_users, n_notes, n_components=n_components)

rating_tensor = torch.FloatTensor(rating_labels.values).to(deconf_mf_model.device)
user_idxs_tensor = torch.LongTensor(user_idxs).to(deconf_mf_model.device)
note_idxs_tensor = torch.LongTensor(note_idxs).to(deconf_mf_model.device)
exp_tensor = torch.ones_like(rating_tensor).to(deconf_mf_model.device)

data = ModelData(rating_tensor, user_idxs_tensor, note_idxs_tensor, exp_tensor)

In [ ]:
train_loss, val_loss = deconf_mf_model.fit(data, epochs=150, lr=0.1, print_interval=20, validate_fraction=0.1, print_loss=True)

In [ ]:
train_loss, val_loss = mf_model.fit(data, epochs=150, lr=0.1, print_interval=20, validate_fraction=0.1, print_loss=True)

# Step 2: Voting Aggregation
Calculate results for different voting aggregation rules.

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
# Define aggregations
def approval(x, dim, threshold=0.7):
    return (x > threshold).float().mean(dim=dim)
quantile = partial(torch.quantile, q=0.25)

# Collect aggregations into dict
filtered_notes = notes[notes['noteId'].isin(filtered_note_ids)]
note_ids = note_encoder.inverse_transform(np.arange(n_notes))
aggs = {'noteId': note_ids}

# Aggregations with deconfounder model
aggs['mean'] = mf_model.get_vote_scores(torch.mean)
aggs['approval'] = mf_model.get_vote_scores(approval)
aggs['quantile'] = mf_model.get_vote_scores(quantile)
#aggs['var'] = mf_model.get_vote_scores(torch.var)

# Aggregations with deconfounder mf model
aggs['decon_mean'] = deconf_mf_model.get_vote_scores(torch.mean)
aggs['decon_approval'] = deconf_mf_model.get_vote_scores(approval)
aggs['decon_quantile'] = deconf_mf_model.get_vote_scores(quantile)
#aggs['var'] = deconf_mf_model.get_vote_scores(torch.var)

In [ ]:
aggs = {k: v.cpu().numpy() for k, v in aggs.items() if k != 'noteId'}


In [ ]:
aggs['noteId'] = note_ids

In [ ]:
note_results = pd.DataFrame(aggs)
scored_notes = filtered_notes.merge(note_results, on='noteId')

In [ ]:
scored_notes

# Evaluation
As a first-pass for evaluation, compare the models (with and without causal inference) in how well they agree with the current Community Notes algorithm. Obviously, eventually we would like to show that we do better than the existing algorithm in some way, so we will need different evaluations down the line, but this is just a quick first pass.

The deconfounded model (that uses causal inference in stage 1) does significantly better in matching the existing algorithm's outputs than a baseline of matrix factorization + voting.

In [ ]:
note_status_df = pd.read_csv(os.path.join(data_path, 'noteStatusHistory-00000.tsv'), sep='\t')

In [ ]:
merged_notes = scored_notes.merge(note_status_df, on='noteId')
misleading_notes = merged_notes[merged_notes['classification'] == 'MISINFORMED_OR_POTENTIALLY_MISLEADING']
misleading_notes

In [ ]:
note_status_key = 'currentStatus'
num_rated_helpful = misleading_notes[note_status_key].value_counts()['CURRENTLY_RATED_HELPFUL']
print(f'Number of notes rated helpful under existing algorithm: {num_rated_helpful}')
agg_keys = list(aggs.keys())
agg_keys.remove('noteId')
for key in agg_keys:
    helpful_notes = misleading_notes.sort_values(key, ascending=False).head(num_rated_helpful)
    num_helpful = helpful_notes[note_status_key].value_counts()['CURRENTLY_RATED_HELPFUL']
    pct_helpful = num_helpful / num_rated_helpful
    print(f'Percentage of CURRENTLY_RATED_HELPFUL notes in top {num_rated_helpful} notes using {key} aggregation: {pct_helpful:.2%}')

In conclusion, the deconfounded model (that uses causal inference in stage 1) seems to do significantly better in matching the existing algorithm's outputs than a baseline of matrix factorization + voting.

## Differences with existing model

### Notes rated helpful under deconfounder model (ours) but not under existing model

In [ ]:
helpful_notes = misleading_notes.sort_values('decon_mean', ascending=False).head(num_rated_helpful)
diff_notes = helpful_notes[helpful_notes[note_status_key] != 'CURRENTLY_RATED_HELPFUL']
diff_notes[['noteId', 'tweetId', 'summary', 'currentStatus', 'currentCoreStatus', 'mostRecentNonNMRStatus', 'lockedStatus', 'decon_mean']]

In [ ]:
diff_notes.columns

### Notes rated helpful under existing model but not under deconfounder model (ours)

In [ ]:
notes_rated_helpful_by_deconf = misleading_notes.sort_values('decon_mean', ascending=False).head(num_rated_helpful)
helpful_note_ids_deconf = notes_rated_helpful_by_deconf['noteId'].values
notes_rated_helpful_by_existing_algo = misleading_notes[misleading_notes[note_status_key] == 'CURRENTLY_RATED_HELPFUL']
diff_notes = notes_rated_helpful_by_existing_algo[~notes_rated_helpful_by_existing_algo['noteId'].isin(helpful_note_ids_deconf)]
diff_notes = diff_notes.sort_values('decon_mean', ascending=False)
diff_notes[['noteId', 'tweetId', 'summary', 'currentStatus', 'currentCoreStatus', 'mostRecentNonNMRStatus', 'lockedStatus', 'decon_mean']].tail(500)